In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16, InceptionV3, MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [2]:
def load_data(root = 'aml-2024-feather-in-focus/'):

    # load labels and corresponding bird species names
    data = np.load(root + 'class_names.npy', allow_pickle=True)
    data = dict(data.item())    
    class_names = pd.DataFrame(data.items(), columns=['name', 'label'])

    # remove first 4 characters for every class and  replace underscores with spaces
    class_names['name'] = class_names['name'].apply(lambda x: x[4:]).apply(lambda x: x.replace('_', ' '))

    # load the paths of the bird images test set
    birds_df = pd.read_csv(root + 'train_images.csv')
    birds_df['image_path'] = birds_df['image_path'].apply(lambda x: root + 'train_images' + x)   

    # merge the labels and image paths with the species names
    birds_df = birds_df.merge(class_names, left_on='label', right_on='label')

    return birds_df, class_names

birds_df, class_names = load_data()
birds_df

,image_path,label,name
0,aml-2024-feather-in-focus/train_images/train_i...,1,Black footed Albatross
1,aml-2024-feather-in-focus/train_images/train_i...,1,Black footed Albatross
2,aml-2024-feather-in-focus/train_images/train_i...,1,Black footed Albatross
3,aml-2024-feather-in-focus/train_images/train_i...,1,Black footed Albatross
4,aml-2024-feather-in-focus/train_images/train_i...,1,Black footed Albatross
...,...,...,...
3921,aml-2024-feather-in-focus/train_images/train_i...,200,Common Yellowthroat
3922,aml-2024-feather-in-focus/train_images/train_i...,200,Common Yellowthroat
3923,aml-2024-feather-in-focus/train_images/train_i...,200,Common Yellowthroat
3924,aml-2024-feather-in-focus/train_images/train_i...,200,Common Yellowthroat


In [4]:
train_df, temp_df = train_test_split(birds_df, test_size=0.4,
                                     shuffle=True,random_state=42, stratify=birds_df['name']
                                    )


val_df, test_df = train_test_split(temp_df, test_size=0.5,
                                     shuffle=True,random_state=42, stratify=temp_df['name']
                                    )

train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')


train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='name',  
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    seed=42
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='name', 
    target_size=(224, 224), 
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    seed=42
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='name',
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False,
    seed=42,
)


Found 2355 validated image filenames belonging to 200 classes.
Found 785 validated image filenames belonging to 200 classes.
Found 786 validated image filenames belonging to 200 classes.


In [ ]:

def get_model(model_name, num_classes=200):
    if model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
        raise ValueError("Model name not recognized")
    
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax') 
    ])
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [34]:
model_name = 'ResNet50' 
# model_name = 'EfficientNetB0'
# model_name = 'VGG16'
# model_name = 'InceptionV3'
# model_name = 'MobileNetV2'

model = get_model(model_name)


94765736/94765736 [==============================] - 16s 0us/step
Epoch 1/10
74/74 [==============================] - 147s 2s/step - loss: 5.4238 - accuracy: 0.0055 - val_loss: 5.2532 - val_accuracy: 0.0064
Epoch 2/10
74/74 [==============================] - 135s 2s/step - loss: 5.2665 - accuracy: 0.0072 - val_loss: 5.2460 - val_accuracy: 0.0127
Epoch 3/10
74/74 [==============================] - 133s 2s/step - loss: 5.2492 - accuracy: 0.0068 - val_loss: 5.2414 - val_accuracy: 0.0076
Epoch 4/10
74/74 [==============================] - 137s 2s/step - loss: 5.2480 - accuracy: 0.0085 - val_loss: 5.2195 - val_accuracy: 0.0115
Epoch 5/10
74/74 [==============================] - 138s 2s/step - loss: 5.2379 - accuracy: 0.0106 - val_loss: 5.2133 - val_accuracy: 0.0153
Epoch 6/10
74/74 [==============================] - 135s 2s/step - loss: 5.2223 - accuracy: 0.0106 - val_loss: 5.2190 - val_accuracy: 0.0089
Epoch 7/10
74/74 [==============================] - 126s 2s/step - loss: 5.2147 - accura

In [ ]:

model.fit(train_generator, epochs=10, validation_data=validation_generator)


In [ ]:

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


In [ ]:


model.save(model_name + '_model.h5')
model.save_weights(model_name + '_weights.h5')
